## 멘토링 피드백
- [12.5] (0.8939)hybrid retrieve에서 sparse비율 늘리고, topk 100개로 먼저 찾은다음에 reranking으로 top5개 구하기 
- [14.1] (0.8848)일단 확인용으로 sentencetransformer로 sementic chunker한거로 hybrid-sparse-boost=0.0025하고 10 개 찾고, 리랭킹 하는 방식으로 일단 확인
- [14.2] opneai로 sementic chuncker 만든거
- [14.3] [14.1]에 동의어만 추가해서 

## 최종 실험 돌리기 
- [15.1] => [12.5]와 [14.1] 합치기 (boost=0.5, k=100)
- [15.2] boost= 0.5 -> 0.2
- 성능이 떨어진 이유: 79 번 보면 "슈퍼옥시드 디스무타아제 라는 효소에 대해 설명해줘." 이 쿼리에서 문서에는 영어로 쓰여져있음. 12.5,14.1은 찾는데 15.1은 못찾음 
- 
- [15.3]dense_retrieve()로 대채 15 개 찾기 (자체:0.82803)
- [15.4]boost값 0.05로해서 topk 15개 (자체: 0.81515)
- [15.5]hybrid - 0.0025 topk 15 , 동의어 삭제 (자체: 0.84470)
- [15.6] boost=0.3 topk=200
- [15.7] hybird_retrieve에서 중복제거 boost값 0.003 topk=10 
- [15.8] [15.7]에서 boost=0.2, topk=20 

- [16.1] chunk랑 원본이랑 합치기 boost 0.2, topk=10
- [16.2] setting바꾸고 document.chunk로 색인, boost=0.003, topk=10 
= [16.3] boost=0.003, 기본 setting, topk=70

# Document Chunking

In [ ]:
# pip install langchain langchain-experimental openai
# pip install tiktoken

In [ ]:
import json
from langchain_experimental.text_splitter import SemanticChunker
from sentence_transformers import SentenceTransformer

# embedding 모델
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

class CustomEmbeddings:
    def embed_documents(self, documents):
        return model.encode(documents)

text_splitter = SemanticChunker(CustomEmbeddings())

output_file = '/data/ephemeral/home/data/document_chunk.jsonl'

with open('/data/ephemeral/home/data/documents.jsonl', 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:

        data = json.loads(line)
        docid = data['docid']
        src = data['src']
        content = data['content']

        chunks = text_splitter.split_text(content)

        for chunk in chunks:
            output_data = {
                "docid": docid,
                "src": src,
                "content": chunk
            }

            outfile.write(json.dumps(output_data, ensure_ascii=False) + '\n')

print(f"청킹 완료 :'{output_file}'")

# RAG Pipeline

In [1]:
#!./elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori

In [2]:
#Suk3UCF_msp1jgipFph-


In [1]:
# 엘라스틱서치의 데몬 인스턴스 만들기
# Nori 설치 이전에 데몬을 생성하면 Nori가 바로 사용할 수 없음, 이때는 데몬 재실행 필요

import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['/data/ephemeral/home/elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(30)


In [2]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon   1277801 1277783  6 05:55 ?        00:00:02 /data/ephemeral/home/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=/data/ephemeral/home/elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/data/ephemeral/home/elasticsearch-8.8.0 -Des.path.conf=/data/ephemeral/home/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /data/ephemeral/home/elasticsearch-8.8.0/lib/*:/data/ephemeral/home/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon   1277860 1277801 99 05:55 ?        00:00:52 /data/ephemeral/home/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -D

In [4]:
username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
password = 'PASSWORD elastic'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/data/ephemeral/home/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': 'instance-11887',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'Qi0bps3KR76S74__mRoyvA',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [5]:
import os
import json
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
# model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [85]:
# def hybrid_retrieve(query_str, size):
#     query_embedding = get_embedding([query_str])[0]
    
#     body = {
#         "query" : {
#             "match" : {
#                 "content": {
#                     "query": query_str,
#                     "boost" : 0.0030 # [4, 4.5] -> 0.0025
#                 }
#             }
#         },
#     "knn": {
#         "field": "embeddings",
#         "query_vector" : query_embedding.tolist(),
#         "k" : 5,
#         "num_candidates" : 50,
#         "boost": 1
#     },
#     "size": size 
#     }
#     return es.search(index="test", body = body)

In [131]:
# 역색인 + Vector 유사도 혼합
def hybrid_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]
    body = {
        "query": {
            "match": {
                "content": {
                    "query": query_str,
                    # "boost": 0.0030
                    "boost" : 0.0025 
                }
            }
        },
        "knn": {
            "field": "embeddings",
            "query_vector": query_embedding.tolist(),
            "k": 5,
            "num_candidates": 50,
            "boost": 1
        },
        "collapse":{
            "field": "docid.keyword",  # keyword 타입으로 설정된 필드 사용
            "inner_hits": {
                "name": "top_hits",
                "size": 1,
                "sort": [
                    {"_score": "desc"}
                    ]
                }
            },
        "size": size
    }
    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", body=body)

In [132]:

# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        # 역색인 필드 
        "content": {"type": "text", "analyzer": "nori"}, 
        # 임베딩 필드 
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

In [120]:
# # 색인을 위한 setting 설정
# settings = {
#     "analysis": {
#         "analyzer": {
#             "nori": {
#                 "type": "custom",
#                 "tokenizer": "nori_tokenizer",
#                 "decompound_mode": "mixed", 
#                 # "filter": ["nori_posfilter"]
#                 "filter": ["nori_posfilter", "synonyms_filter"]
#             }
#         },
#         "filter": {
#             "nori_posfilter": {
#                 "type": "nori_part_of_speech",
#                 # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
#                 "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX", "XSA"]
#                             #  ["IC", "MAG", "MAJ", "MM", "SP", "SSC", "SSO", "XPN", "XSA", "XSN", "XSV", "UNA", "NA", "VSV" ]
#             },
#             "synonyms_filter": {
#                 "type": "synonym",
#                 "lenient": True,
#                 "synonyms": [
#                     "복숭아, 복숭아나무",
#                     "밀물, 만조",
#                     "썰물, 간조",
#                     "관계, 서로 연결, 긴밀",
#                     "순기능, 도움",
#                     "리보오솜, 리보솜",
#                     "네트웍, 네트워크",
#                     "역할, 기능, 기여, 담당",
#                     "전력, 전류",
#                     "이루어져, 구성, 형성",
#                     "땅, 토양",
#                     "가장 많이, 대부분",
#                     "단점, 부정적인",
#                     "원자력 발전, 에너지",
#                     "형태, 유형",
#                     "원자, 가장 작은 입자",
#                     "내부 구조, 구성"
#                     "자연, 생태계",
#                     "뭉쳐, 모여",
#                     "날씨, 기후",
#                     "측정, 추정",
#                     "비만도, 체중 상태",
#                     "관계, 영향",
#                     "뭐야, 의미",
#                     "해안, 바다",
#                     "전구, 램프",
#                     "번식, 생식",
#                     "작은 기체 하나, 기체의 분자",
#                     "다음 세대, 후손",
#                     "무게, 질량",
#                     "잠, 수면",
#                     "약, 약물",
#                     "자석, 자성체",
#                     "동물, 생물체",
#                     "누가, 비해",
#                     "결혼 전, 혼전",
#                     "숨, 호흡",
#                     "인터페론, interferon", ## 여기서부터 새로 추가
#                     "트로이군, trojan",
#                     "슈퍼옥시드 디스무타아제, Superoxide Dismutase",
#                     "드미트리 이바노프스키, Dmitri Ivanovsky",
#                     "카탈로그, catalog",
#                     "파이썬, python",
#                     "디엔에이, DNA",
#                     "이란 콘트라, 이란-콘트라",
#                     "병합 정렬, merge sort",
#                     "샘플, sample",
#                     "연산자, operator",
#                     "칼리시바이러스,칼리시 바이러스",
#                     "브리지 인버터, bridge inverter",
#                     "기억 상실증, 기억 상실, 기억력 저하",
#                     "음파, 파",
#                     "통학 버스, 학교 버스",
#                     "극대화, 가장 큰",
#                     "태양광을 에너지로 변환하는 능력, 광합성",
#                     "자라다, 자랄",
#                     "스모그, 매연",
#                     "동일한, 하나의",
#                     "나이가 들수록, 노인들의",
#                     "학습된, 사회화",
#                     "탄소의 내부 구조, 탄소 원자의 구조",
#                     "생존의 확률, 생존률",
#                 ]
#             }
#         }
#     },
#     "index": {
#         "similarity": {
#             "lm_jelinek_mercer": { 
#                 "type": "LMJelinekMercer", 
#                 "lambda": 0.7
#             } 
#         }
#     }
# }

# # 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
# mappings = {
#     "properties": {
#         "content": {
#             "type": "text", 
#             "analyzer": "nori",
#             "similarity": "lm_jelinek_mercer"
#         }, 
#         "embeddings": {
#             "type": "dense_vector",
#             "dims": 768,
#             # "dims": 1024,
#             "index": True,
#             "similarity": "l2_norm"
#             # "similarity": "cosine"
#         }
#     }
# }

In [121]:
create_es_index("test", settings, mappings)

In [122]:
# Document Chunking content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/document_chunk.jsonl") as f:
#with open("/data/ephemeral/home/data/merged_documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

""" 
docs[0] -> 
{'docid': '42508ee0-c543-4338-878e-d98c6babee66',
 'src': 'ko_mmlu__nutrition__test',
 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. ....
 }
"""

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
batch 4300
batch 4400
batch 4500
batch 4600
batch 4700
batch 4800
batch 4900
batch 5000
batch 5100
batch 5200
batch 5300
batch 5400
batch 5500
batch 5600
batch 5700
batch 5800
batch 5900
batch 6000
batch 6100
batch 6200
batch 6300
batch 6400
batch 6500
batch 6600
batch 6700
batch 6800
batch 6900
batch 7000
batch 7100
batch 7200
batch 7300
batch 7400
batch 7500
batch 7600
batch 7700
batch 7800
batch 7900
batch 8000
batch 8100
batch 8200
batch 8300
batch 8400
batch 8500


" \ndocs[0] -> \n{'docid': '42508ee0-c543-4338-878e-d98c6babee66',\n 'src': 'ko_mmlu__nutrition__test',\n 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. ....\n }\n"

In [123]:
# doc = docs[0], len(embedding) = 768
# doc_key = docid, src, content
for doc, embedding in zip(docs, embeddings):  
    doc['embeddings'] = embedding.tolist() 
    index_docs.append(doc)
    
# index_docs_key = docid, src, content, embeddings 

In [124]:
ret = bulk_add("test", index_docs)

In [125]:
ret

(8540, [])

In [114]:
test_query = "네트웍 계층 중 IP에 대해 설명해줘."
# 역색인을 사용하는 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'docid: ',rst["_source"]["docid"], 'source:', rst['_source']["content"])

/tmp/ipykernel_1277783/1430155283.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


score: 0.74526834 docid:  5f206999-c7fc-4252-aa37-27655345e78d source: 패킷 필터 방화벽은 네트워크 또는 전송 계층에서 필터링을 수행합니다. 이 방화벽은 네트워크 트래픽을 모니터링하고, 특정한 규칙에 따라 패킷을 허용하거나 차단합니다. 이러한 필터링 위치는 네트워크와 전송 계층에서 이루어지며, 이를 통해 네트워크 보안을 강화할 수 있습니다. 패킷 필터 방화벽은 IP 주소, 포트 번호, 프로토콜 등 다양한 요소를 기반으로 패킷을 분석하고 처리합니다. 이를 통해 악성 트래픽이나 불필요한 트래픽을 차단하여 네트워크의 안전성을 유지합니다.
score: 0.71210486 docid:  82b28534-6a0d-49df-865c-655c264605a7 source: 계층적 시스템은 개인이 다양한 선택지를 고려하고 이를 순차적으로 평가하여 최적의 선택을 할 수 있는 시스템입니다. 따라서, 합리적 선택 이론은 계층적 시스템에서 가장 정확하게 설명될 수 있습니다.
score: 0.6620711 docid:  ee649cd9-7406-4227-abf0-87c14288b3f8 source: 무차별 대입 공격은 실패한 세션에서 액세스 자격 증명을 허용합니다. 이러한 유형의 공격은 OSI 모델의 세션 계층에서 수행됩니다. 세션 계층은 네트워크 통신의 세션을 설정, 유지 및 해제하는 역할을 담당합니다. 따라서 액세스 자격 증명에 대한 무차별 대입 공격은 세션 계층에서 발생할 수 있습니다.
score: 0.60458696 docid:  2fa73480-d1ad-48d8-925a-f9ad8fe4d753 source: SNMP 열거를 수행하는 데 사용하는 도구는 IP 네트워크 브라우저입니다. IP 네트워크 브라우저는 SNMP 프로토콜을 사용하여 네트워크 장치의 정보를 수집하고 모니터링하는 데 사용됩니다. 이 도구는 SNMP 에이전트가 설치된 장치에 연결하여 장치의 상태, 성능 및 구성과 관련된 정보를 검색할 수 있습니다. IP 네트워

In [89]:
test_query = "네트워크 분석을 위한 traffic sniffing이 뭐야?"
# 역색인을 사용하는 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'docid: ',rst["_source"]["docid"], 'source:', rst['_source']["content"])

score: 0.05274108 docid:  f75d3717-24c8-41e6-a78d-ea6f92444209 source: 에터피크는 다양한 프로토콜 네트워크에서 네트워크 분석에 널리 사용되는 도구입니다. 이 도구는 네트워크 트래픽을 모니터링하고 분석하여 네트워크의 동작과 문제를 파악할 수 있습니다. 에터피크는 패킷 스니핑 기능을 제공하여 네트워크 상에서 전송되는 데이터를 실시간으로 캡처하고 분석할 수 있습니다. 또한, 에터피크는 다양한 프로토콜을 지원하여 TCP/IP, 이더넷, Wi-Fi 등 다양한 네트워크 환경에서 사용할 수 있습니다.
score: 0.046941318 docid:  7c75080c-5d8f-41e2-ad26-c90ad3f671b0 source: 무선 트래픽 스니핑은 포렌식 조사 시 또는 무선 문제 해결 중 도움이 될 수 있는 무선 트래픽 분석 과정입니다. 이 과정은 무선 네트워크에서 전송되는 데이터를 감시하고 분석하여 문제를 해결하거나 증거를 수집하는 데 사용됩니다. 무선 트래픽 스니핑은 무선 네트워크의 보안 취약점을 식별하고 해결하는 데 도움이 됩니다.
score: 0.041011 docid:  1777c375-14e7-474b-a5c6-6457798fe2de source: 핑 스윕은 네트워크 관리자가 네트워크 상태를 확인하고 활성 시스템을 식별하기 위해 사용하는 도구입니다. 이 도구는 네트워크에 연결된 장치들과의 통신을 테스트하여 응답 여부를 확인합니다. 핑 스윕은 네트워크의 문제를 신속하게 진단하고 해결하기 위해 필수적인 도구입니다. 네트워크 관리자는 핑 스윕을 사용하여 네트워크의 성능을 모니터링하고 장애 발생 시 신속하게 대응할 수 있습니다. 또한, 핑 스윕은 네트워크에 연결된 장치들의 상태를 확인하여 활성 시스템을 식별하는 데에도 사용됩니다.
score: 0.033637106 docid:  37550e6e-38f5-4e5f-8eec-f0a42e485dee source: 두 번째로, 극점과 영점이 모두 음의 실수 축에 배치되는 

/tmp/ipykernel_1277783/2417144229.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


## RAG 구현 

In [15]:
# - 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
# - 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.

In [16]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 지식에 관한 주제로 질문하면 반드시 search api를 호출할 수 있어야 한다.
- 지식과 관련되지 않은 나머지 대화 메시지에는 함수 호출 없이 적절한 대답을 생성한다.
"""



In [17]:
# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "long_question": {
                        "type": "string",
                        # "description": "Final query suitable for use in search from the user messages history"
                        # "description": "Keywords suitable for use in search engine."
                        # "description": "User's question in Korean, including all the keywords in the user messages"
                        "description": "User's question in Korean. Full message if the user message is single-turn."}
                        #"description" : "User's question in Korean. Full message if the user message is single-turn. Additionally, categorize the question and infer user's intent."}
                },
                "required": ["long_question"],
                "type": "object"
            }
        }
    },
]

In [28]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
import json

# Ranker 모델 로드
model_path = 'Dongjin-kr/ko-reranker'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model_reranker = AutoModelForSequenceClassification.from_pretrained(model_path)
model_reranker.eval()

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

In [126]:
# 상위 3개 

def rerank_with_model(query, retrieved_docs):
    # Query와 검색된 문서를 쌍으로 만듬
    pairs = [[query, doc["_source"]["content"]] for doc in retrieved_docs]
    
    # Ranker 모델을 통해 점수 계산
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model_reranker(**inputs, return_dict=True).logits.view(-1, ).float()
        normalized_scores = exp_normalize(scores.numpy())

    # 점수에 따라 문서를 재정렬하고 상위 3개의 문서만 남김
    reranked_docs = sorted(zip(retrieved_docs, normalized_scores), key=lambda x: x[1], reverse=True)[:5]
    return reranked_docs

In [133]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출 후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("long_question")

        # 검색 결과 추출
        # search_result = hybrid_retrieve(standalone_query, 100)
        search_result = hybrid_retrieve(standalone_query, 10)
        # 검색 결과를 Ranker 모델로 재정렬
        reranked_docs = rerank_with_model(standalone_query, search_result['hits']['hits'])

        response["standalone_query"] = standalone_query
        retrieved_context = []
        
        # 중복을 방지하기 위한 set
        #seen_docids = set()
        
        for doc, score in reranked_docs:
            #docid = doc["_source"]["docid"]
            # docid가 이미 추가된 경우는 생략
            #if docid not in seen_docids:
            retrieved_context.append(doc["_source"]["content"])
            response["topk"].append(doc["_source"]["docid"])
            response["references"].append({"score": float(score), "content": doc["_source"]["content"]})
                #seen_docids.add(docid)  # 이미 처리한 docid를 set에 추가
                
        # 재정렬된 문서들을 LLM에 다시 넣어 답변 생성
        # messages.append({"role": "assistant", "content": json.dumps(retrieved_context)})
        # msg = [{"role": "system", "content": persona_qa}] + messages
        # try:
        #     qaresult = client.chat.completions.create(
        #             model=llm_model,
        #             messages=msg,
        #             temperature=0,
        #             seed=1,
        #             timeout=30
        #         )
        # except Exception as e:
        #     traceback.print_exc()
        #     return response
        # response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response


In [134]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../data/eval.jsonl", "../submission/[16.3]submission.csv")


Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/tmp/ipykernel_1277783/1357278640.py:36: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body=body)


Answer: 

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]
Answer: 

Test 5
Question: [{'role': 'user', 'content': '피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?'}]
Answer: 

Test 6
Question: [{'role': 'user', 'content': '헬륨이 다른 원소들과 반응을 잘 안하는 이유는?'}]
Answer: 

Test 7
Question: [{'role': 'user', 'content': '문맹 비율이 사회 발전에 미치는 영향은?'}]
Answer: 

Test 8
Question: [{'role': 'user', 'content': '이란 콘트라 사건이 뭐야'}, {'role': 'assistant', 'content': '이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.'}, {'role': 'user', 'content': '이 사건이 미국 정치에 미친 영향